##Train notebook
Ниже представлен цикл обучения `CycleGan` на `GoogleColabortory`. Если хотите использовать его для обучения - следуйте комментариям ниже. И затем нажмите `ctrl+f9`

In [ ]:
# в FILE укажите, какой датасет будете использовать
# на выбор: apple2orange, summer2winter_yosemite, horse2zebra, monet2photo,
# cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades,
# iphone2dslr_flower, ae_photos

FILE = "summer2winter_yosemite"

# если в FILE используете датасет, отличный от тех, что ниже, то не забудьте его добавить
if FILE == "summer2winter_yosemite": prefix = "sum2win"

# в drive_path укажите путь к директории с проектм, чтобы все данные сохранялись там
drive_path = "/content/drive/MyDrive/TelegramStyleBot/"
path_to_dicts = drive_path + "st_dicts/"
path_to_conf = drive_path + f"train/configs/{prefix}_losses.json"

# будете ли вы сохранять и использовать данные с диска
use_drive = True

# инициализировать ли веса модели заново, если нет,
# то они будут загружены из словарей состояния
init_model = False

# иногда дискриминатор может сталть слишком сильным,
# поэтому можно обновлять его веса только каждые n эпох обучения
skip_disc = False; skip_step = 3

n_epochs = 2; batch_size = 2; dataroot = f"datasets/{FILE}/";
lr = 1e-4; size = 256; input_nc = 3; output_nc = 3;
id_loss_coef = 0.3; gan_loss_coef = 1.5; cycle_loss_coef = 10
gen_lr_coef = 1; disc_lr_coef = 1;

In [2]:
from google.colab import drive
import time

if use_drive:
  for _ in range(2):
    try:
      drive.mount('/content/drive')
      break
    except:
      time.sleep(60)

Mounted at /content/drive


In [3]:
!pip install torchmetrics && pip install ujson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
import itertools
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.models import vgg16
from torch.utils.data import Dataset, DataLoader
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

psnr_metric = PeakSignalNoiseRatio().to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
import os
import ujson

Using device: cuda


In [5]:
!rm -rf ./datasets

In [6]:
URL = f"http://efrosgans.eecs.berkeley.edu/cyclegan/datasets/{FILE}.zip"
ZIP_FILE = f"./datasets/{FILE}.zip"
TARGET_DIR = f"./datasets/{FILE}"

!mkdir -p ./datasets

!wget -N "{URL}" -O "{ZIP_FILE}"

!unzip -q "{ZIP_FILE}" -d ./datasets/
!rm "{ZIP_FILE}"

!mkdir -p "{TARGET_DIR}/train/A" "{TARGET_DIR}/train/B" "{TARGET_DIR}/test/A" "{TARGET_DIR}/test/B"

!mv "{TARGET_DIR}/trainA/"* "{TARGET_DIR}/train/A/"
!mv "{TARGET_DIR}/trainB/"* "{TARGET_DIR}/train/B/"
!mv "{TARGET_DIR}/testA/"* "{TARGET_DIR}/test/A/"
!mv "{TARGET_DIR}/testB/"* "{TARGET_DIR}/test/B/"

!rm -rf "{TARGET_DIR}/trainA"
!rm -rf "{TARGET_DIR}/trainB"
!rm -rf "{TARGET_DIR}/testA"
!rm -rf "{TARGET_DIR}/testB"

for details.

--2025-07-05 13:36:37--  http://efrosgans.eecs.berkeley.edu/cyclegan/datasets/summer2winter_yosemite.zip
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132640856 (126M) [application/zip]
Saving to: ‘./datasets/summer2winter_yosemite.zip’

./datasets/summer2w 100%[===================>] 126.50M  3.34MB/s    in 32s     

2025-07-05 13:37:09 (3.91 MB/s) - ‘./datasets/summer2winter_yosemite.zip’ saved [132640856/132640856]



In [7]:
!ls -l

total 12
drwxr-xr-x 3 root root 4096 Jul  5 13:37 datasets
drwx------ 5 root root 4096 Jul  5 13:34 drive
drwxr-xr-x 1 root root 4096 Jul  1 21:04 sample_data


In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [  nn.Conv2d(in_features, in_features, 3, padding=1, padding_mode='reflect'),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.Conv2d(in_features, in_features, 3, padding=1, padding_mode='reflect'),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True) ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, device, input_nc, output_nc, n_du_samples=1, n_residual_blocks=2, hidden_size=96):
        super(Generator, self).__init__()

        self.device = device

        model = [   nn.Conv2d(input_nc, hidden_size, 7, padding=3, padding_mode='reflect'),
                    nn.InstanceNorm2d(hidden_size),
                    nn.ReLU(inplace=True) ]

        in_features = hidden_size
        out_features = in_features*2
        for _ in range(n_du_samples):
            model += [  nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features*2

        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        out_features = in_features//2
        for idx in range(n_du_samples):
            upsample_block = [
                nn.Conv2d(in_features, out_features * 4, kernel_size=3, padding=1, padding_mode='reflect'),
                nn.PixelShuffle(2),
                # nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]

            model += upsample_block
            in_features = out_features
            out_features = in_features // 2

        model += [  nn.Conv2d(hidden_size, output_nc, 7, padding=3, padding_mode='reflect'),
                    nn.Tanh() ]

        self.model = nn.Sequential(*model)
        self.model.apply(self.init_weights)

    def forward(self, x):
        return self.model(x).to(self.device)

    def init_weights(self, m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            nn.init.normal_(m.weight, 0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

class Discriminator(nn.Module):
    def __init__(self, device, input_nc, hidden_n:int=2, hidden_size:int=64):
        super(Discriminator, self).__init__()
        self.device = device
        h = hidden_size

        model = [   nn.Conv2d(input_nc, h, 4, stride=2, padding=1),
                    nn.LeakyReLU(0.2, inplace=True) ]

        for _ in range(hidden_n):
            h_next = h*2
            model += [
                    nn.Conv2d(h, h_next, 4, stride=2, padding=1),
                    nn.InstanceNorm2d(h_next),
                    nn.LeakyReLU(0.2, inplace=True)
                ]
            h = h_next

        model += [nn.Conv2d(h, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)
        self.model.apply(self.init_weights)

    def forward(self, x):
        x =  self.model(x)
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1).to(self.device)

    def init_weights(self, m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            nn.init.normal_(m.weight, 0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

In [9]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode='train'):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, mode, 'A') + '/*.*'))
        self.files_B = sorted(glob.glob(os.path.join(root, mode, 'B') + '/*.*'))

        if len(self.files_A) == 0:
            raise RuntimeError(f"No files found in {os.path.exists(os.path.join(root, mode, 'A'))}")
        if len(self.files_B) == 0:
            raise RuntimeError(f"No files found in {os.path.exists(os.path.join(root, mode, 'B'))}")

    def __getitem__(self, index):
        item_A = self.transform(Image.open(self.files_A[index % len(self.files_A)]).convert('RGB'))

        if self.unaligned:
            item_B = self.transform(Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)]).convert('RGB'))
        else:
            item_B = self.transform(Image.open(self.files_B[index % len(self.files_B)]).convert('RGB'))

        return {'A': item_A, 'B': item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [10]:
class ReplayBuffer:
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.buffer = []  # [1, C, H, W]

    def push_and_pop(self, data: torch.Tensor) -> torch.Tensor:
        """Принимает/возвращает тензоры в формате [N, C, H, W]"""
        outputs = []

        for image in data:
            image = image.unsqueeze(0)  # [C,H,W] -> [1,C,H,W]

            if len(self.buffer) < self.max_size:
                self.buffer.append(image)
                outputs.append(image)
            else:
                if random.random() > 0.5:  # 50% возвращаем из буфера
                    idx = random.randint(0, self.max_size - 1)
                    outputs.append(self.buffer[idx].clone())
                    self.buffer[idx] = image  # Заменяем старый элемент
                else:
                    outputs.append(image)  # Возвращаем новый элемент

        # Все в один тензор [K, C, H, W]
        return torch.cat(outputs, dim=0)


In [11]:
def load_or_create_config(path, sub_conf):
    try:
        with open(path, "r") as file:
            return ujson.load(file)
    except:
        with open(path, "w") as file:
            ujson.dump(sub_conf, file)
        return sub_conf

def load_to_conf(path_to_conf:str, sub_conf:dict, temp_list:list):
  target = load_or_create_config(path_to_conf, sub_conf)

  for _ in range(2):
    try:
      for i, key in enumerate(sub_conf):
          target[key].append(temp_list[i])
      break
    except:
      target.update(sub_conf)


  with open(path_to_conf, "w") as file:
      ujson.dump(target, file)

# prefix = "net"
# path_to_conf_dir = "/content/confs/"

# path_to_conf = path_to_conf_dir + prefix + "_conf.json"


# sub_conf = {"1": [], "2": []}

# first = 1
# second = 2
# temp_list = [first, second]

# load_to_conf(path_to_conf, sub_conf, temp_list)


# with open(path_to_conf, "r") as file:
#     main_conf = ujson.load(file)
#     print(main_conf)

def compute_metrics(real, generated, recovered):
    with torch.no_grad():
        real_01 = (real + 1) / 2
        gen_01 = (generated + 1) / 2
        rec_01 = (recovered + 1) / 2

        psnr_val = psnr_metric(gen_01, real_01)
        ssim_val = ssim_metric(gen_01, real_01)
        cycle_psnr = psnr_metric(rec_01, real_01)
        cycle_ssim = ssim_metric(rec_01, real_01)

    return psnr_val.item(), ssim_val.item(), cycle_psnr.item(), cycle_ssim.item()


In [16]:
netG_A2B = Generator(device, input_nc, output_nc).to(device)
print(netG_A2B)
netG_B2A = Generator(device, output_nc, input_nc).to(device)

netD_A = Discriminator(device, input_nc).to(device)
netD_B = Discriminator(device, output_nc).to(device)

def load_model(model, filename):
    model_path = path_to_dicts + filename
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Модель {filename} успешно загружена")

def save_model(model, path_to_dicts):
    model.to('cpu')
    torch.save(model.state_dict(), path_to_dicts)
    model.to(device)

if not init_model:
  load_model(netG_A2B, f'{prefix}_netG_A2B.pth')
  load_model(netG_B2A, f'{prefix}_netG_B2A.pth')
  load_model(netD_A, f'{prefix}_netD_A.pth')
  load_model(netD_B, f'{prefix}_netD_B.pth')

criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr=(lr*gen_lr_coef), betas=(0.5, 0.999))

optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=(lr*disc_lr_coef), betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=(lr*disc_lr_coef), betas=(0.5, 0.999))

transforms_ = [ transforms.Resize(int(size*1.12), Image.BICUBIC),
                transforms.RandomCrop(size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(dataroot, transforms_=transforms_, unaligned=True),
                        batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

Generator(
  (model): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): InstanceNorm2d(96, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(192, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): ResidualBlock(
      (conv_block): Sequential(
        (0): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(192, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
        (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (4): InstanceNorm2d(192, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (5): ReLU(inplace=True)
      )
    )
    (7): Re

In [ ]:
Tensor = torch.cuda.FloatTensor if device == 'cuda' else torch.Tensor
input_A = Tensor(batch_size, input_nc, size, size)
input_B = Tensor(batch_size, output_nc, size, size)
target_real = torch.ones(batch_size, 1, device=device, requires_grad=False)
target_fake = torch.zeros(batch_size, 1, device=device, requires_grad=False)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

losses_dict = {
    'GA_Loss': [],
    'GB_Loss': [],
    'DA_Loss': [],
    'DB_Loss': [],
    'PSNR_avg': [],
    'SSIM_avg': [],
    'Cycle_PSNR_avg': [],
    'Cycle_SSIM_avg': [],
}

for epoch in range(n_epochs):
  gen_loss = 0
  ga_loss = 0
  gb_loss = 0
  da_loss = 0
  db_loss = 0

  psnr_total = 0.0
  ssim_total = 0.0
  cycle_psnr_total = 0.0
  cycle_ssim_total = 0.0
  metric_count = 0

  for i, batch in enumerate(dataloader):

    real_A = batch['A'].to(device, dtype=torch.float32)
    real_B = batch['B'].to(device, dtype=torch.float32)

    ###### Gens A2B & B2A ######
    optimizer_G.zero_grad()

    # Identity loss
    same_B = netG_A2B(real_B)
    loss_identity_B = criterion_identity(same_B, real_B)*id_loss_coef

    same_A = netG_B2A(real_A)
    loss_identity_A = criterion_identity(same_A, real_A)*id_loss_coef

    # GAN loss
    fake_B = netG_A2B(real_A)
    pred_fake = netD_B(fake_B) #(1, 1)
    loss_GAN_A2B = criterion_GAN(pred_fake, target_real)*gan_loss_coef

    fake_A = netG_B2A(real_B)
    pred_fake = netD_A(fake_A)
    loss_GAN_B2A = criterion_GAN(pred_fake, target_real)*gan_loss_coef

    imgs = {
    'real_A': real_A, 'fake_B': fake_B, 'real_B': real_B, 'fake_A': fake_A
    }

    # Cycle loss
    recovered_A = netG_B2A(fake_B)
    loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*cycle_loss_coef

    recovered_B = netG_A2B(fake_A)
    loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*cycle_loss_coef

    # Total loss
    loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
    gen_loss += loss_G.item()
    loss_G.backward()

    ga_loss += (loss_identity_B + loss_GAN_A2B).item()
    gb_loss += (loss_identity_A + loss_GAN_B2A).item()

    optimizer_G.step()
    torch.cuda.empty_cache()

    ###### Disc A ######

    # Real loss
    pred_real = netD_A(real_A)
    loss_D_real = criterion_GAN(pred_real, target_real)

    # Fake loss
    fake_A = fake_A_buffer.push_and_pop(fake_A)
    pred_fake = netD_A(fake_A.detach())
    loss_D_fake = criterion_GAN(pred_fake, target_fake)

    # Total loss
    loss_D_A = (loss_D_real + loss_D_fake)*0.5
    da_loss += loss_D_A.item()

    if ((epoch+1) % skip_step == 0) or (not skip_disc):
      optimizer_D_A.zero_grad()
      loss_D_A.backward()
      optimizer_D_A.step()
      torch.cuda.empty_cache()

    ###### Disc B ######

    # Real loss
    pred_real = netD_B(real_B)
    loss_D_real = criterion_GAN(pred_real, target_real)

    # Fake loss
    fake_B = fake_B_buffer.push_and_pop(fake_B)
    pred_fake = netD_B(fake_B.detach())
    loss_D_fake = criterion_GAN(pred_fake, target_fake)

    # Total loss
    loss_D_B = (loss_D_real + loss_D_fake)*0.5
    db_loss += loss_D_B.item()

    if ((epoch+1) % skip_step == 0) or (not skip_disc):
      optimizer_D_B.zero_grad()
      loss_D_B.backward()
      optimizer_D_B.step()
      torch.cuda.empty_cache()

    if i % 10 == 0:
            psnr_a, ssim_a, cycle_psnr_a, cycle_ssim_a = compute_metrics(real_A, fake_B, recovered_A)

            psnr_b, ssim_b, cycle_psnr_b, cycle_ssim_b = compute_metrics(real_B, fake_A, recovered_B)

            psnr_total += (psnr_a + psnr_b) / 2
            ssim_total += (ssim_a + ssim_b) / 2
            cycle_psnr_total += (cycle_psnr_a + cycle_psnr_b) / 2
            cycle_ssim_total += (cycle_ssim_a + cycle_ssim_b) / 2
            metric_count += 1

  ga_loss /= len(dataloader)
  gb_loss /= len(dataloader)
  da_loss /= len(dataloader)
  db_loss /= len(dataloader)

  avg_psnr = psnr_total / metric_count if metric_count > 0 else 0
  avg_ssim = ssim_total / metric_count if metric_count > 0 else 0
  avg_cycle_psnr = cycle_psnr_total / metric_count if metric_count > 0 else 0
  avg_cycle_ssim = cycle_ssim_total / metric_count if metric_count > 0 else 0

  temp_list = [ga_loss, gb_loss, da_loss, db_loss, avg_psnr, avg_ssim, avg_cycle_psnr, avg_cycle_ssim]
  load_to_conf(path_to_conf, losses_dict, temp_list)

  if (epoch+1) % 2 == 0:
      keys = ['real_A', 'fake_B', 'real_B', 'fake_A']

      fig, axs = plt.subplots(2, 2, figsize=(12, 12))
      axs = axs.flatten()

      for j, key in enumerate(keys):
        axs[j].imshow((imgs[key].cpu().detach()[0].clip(-1,1).permute(1, 2, 0) + 1) / 2)
        axs[j].set_title(key)
        axs[j].axis('off')

      plt.show()

      save_model(netG_A2B, f'{path_to_dicts}{prefix}_netG_A2B.pth')
      save_model(netG_B2A, f'{path_to_dicts}{prefix}_netG_B2A.pth')
      save_model(netD_A, f'{path_to_dicts}{prefix}_netD_A.pth')
      save_model(netD_B, f'{path_to_dicts}{prefix}_netD_B.pth')

with open(path_to_conf, "r") as file:
  losses = ujson.load(file)
  fig, axs = plt.subplots(2, 2, figsize=(15, 10))

  axs[0, 0].plot(losses['GA_Loss'], label='Gen_A')
  axs[0, 0].plot(losses['GB_Loss'], label='Gen_B')
  axs[0, 0].set_title("Generator Losses")
  axs[0, 0].legend()

  axs[0, 1].plot(losses['DA_Loss'], label='Dis_A')
  axs[0, 1].plot(losses['DB_Loss'], label='Dis_B')
  axs[0, 1].set_title("Discriminator Losses")
  axs[0, 1].legend()

  axs[1, 0].plot(losses['PSNR_avg'], label='PSNR')
  axs[1, 0].plot(losses['Cycle_PSNR_avg'], label='Cycle_PSNR')
  axs[1, 0].set_title("PSNR Metrics")
  axs[1, 0].legend()

  axs[1, 1].plot(losses['SSIM_avg'], label='SSIM')
  axs[1, 1].plot(losses['Cycle_SSIM_avg'], label='Cycle_SSIM')
  axs[1, 1].set_title("SSIM Metric")
  axs[1, 1].legend()

  plt.tight_layout()
  plt.show()